<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo">
</center>

# Practice Lab: Data type considerations for real world datasets

Estimated time needed: 30 minutes

## Objectives

Using this R notebook you will:

1.  Understand two real world datasets
2.  Create tables in SQLite database while paying particular attention to data types
3.  Load the datasets into two separate tables in a SQLite database
4.  Practice your SQL skills in R to solve some problems


# Understand the datasets

To complete the assignment problems in this notebook you will be using subsetted snapshots of one dataset from Statistics Canada, and one from the Bank of Canada. The links to the prepared datasets are provided in the next section; the interested student can explore the landing pages for the source datasets as follows:

1.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2023-01-01&pid=3210035901">Canadian Principal Crops (Data & Metadata)</a>
2.  <a href="https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2023-01-01">Bank of Canada daily average exchange rates</a>

### 1. Canadian Principal Crops Data

This dataset contains agricultural production measures for the principle crops grown in Canada, including a breakdown by province and teritory, for each year from 1908 to 2020.

For this assignment you will use a preprocessed snapshot of this dataset (see next section).

A detailed description of this dataset can be obtained from the StatsCan Data Portal at:
[https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2023-01-01&pid=3210035901&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-RP0203EN-SkillsNetwork-23863830&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)  
Detailed information is included in the metadata file and as header text in the data file, which can be downloaded - look for the 'download options' link.  

### 2. Bank of Canada daily average exchange rates

This dataset contains the daily average exchange rates for multiple foreign currencies. Exchange rates are expressed as 1 unit of the foreign currency converted into Canadian dollars. It includes only the latest four years of data, and the rates are published once each business day by 16:30 ET.

For this assignment you will use a snapshot of this dataset with only the USD-CAD exchange rates included (see next section).

A brief description of this dataset and the original dataset can be obtained from the Bank of Canada Data Portal at:
[https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates/](https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2023-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-RP0203EN-SkillsNetwork-23863830&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)


### Dataset URLs

1.  Annual Crop Data: <https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Practice%20Assignment/Annual_Crop_Data.csv>

2.  Daily FX Data: <https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Practice%20Assignment/Daily_FX.csv>

<span style="color:red">**IMPORTANT:**</span> You will be loading these datasets directly into R data frames from the URLs instead of from the StatsCan and Bank of Canada portals. The versions provided at these URLs are simplified and subsetted versions of the original datasets.


### Load the csv files into dataframes and inspect them

Check the datatypes and whether the data make sense given the column names. In particular, ensure the date columns are typed as \<chr>. 

In R and Python, for example, date-like strings may be interpreted by I/O functions as dates by default. Normally this is fine but keep in mind it may also have unintended consequences, such as when moving data from one environment to another.
  


#### Now let's load these datasets into four separate tables.

Let's first load the RSQLite package:


In [3]:
install.packages("RSQLite")


also installing the dependencies ‘fastmap’, ‘bit’, ‘rlang’, ‘cachem’, ‘bit64’, ‘blob’, ‘memoise’, ‘cpp11’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [1]:
library("RSQLite")

### Connect to the database

Load the RSQLite database interface package and establish a connection. 


In [2]:
# write your solution here.
conn <- dbConnect(RSQLite::SQLite(),"FinalDB_lab4.sqlite")

<details>
<summary>Click here to view/hide solution</summary>
<p>

```
conn <- dbConnect(RSQLite::SQLite(),"FinalDB_lab4.sqlite")
```

</details>


## Table creation steps:

Use the following names for the tables: 

1.  **CROP_DATA**
2.  **DAILY_FX**


### Create "CROP_DATA" table in RSQLite.

Also, check whether the table was successfully created.


In [5]:
#CROP_DATA:
df1 <- dbExecute(conn, 
                    "CREATE TABLE CROP_DATA (
                                      CD_ID INTEGER NOT NULL,
                                      YEAR DATE NOT NULL,
                                      CROP_TYPE VARCHAR(20) NOT NULL,
                                      GEO VARCHAR(20) NOT NULL, 
                                      SEEDED_AREA INTEGER NOT NULL,
                                      HARVESTED_AREA INTEGER NOT NULL,
                                      PRODUCTION INTEGER NOT NULL,
                                      AVG_YIELD INTEGER NOT NULL,
                                      PRIMARY KEY (CD_ID)
                                      )", 
                    errors=FALSE
                    )

    if (df1 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }


ERROR: Error: table CROP_DATA already exists


### Create "DAILY_FX" table in RSQLite.


In [5]:
# DAILY_FX: 
# write your solution here.
df3 <- dbExecute(conn, "CREATE TABLE DAILY_FX (
                                DFX_ID INTEGER NOT NULL,
                                DATE DATE NOT NULL, 
                                FXUSDCAD FLOAT(6),
                                PRIMARY KEY (DFX_ID)
                                )",
                    errors=FALSE
                    )

    if (df3 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    } 

Table was created successfully.


<details>
<summary>Click here to view/hide solution</summary>
<p>

```
df3 <- dbExecute(conn, "CREATE TABLE DAILY_FX (
                                DFX_ID INTEGER NOT NULL,
                                DATE DATE NOT NULL, 
                                FXUSDCAD FLOAT(6),
                                PRIMARY KEY (DFX_ID)
                                )",
                    errors=FALSE
                    )

    if (df3 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    } 
```

</details>


### Load the dataframes into the SQLite Database tables you created.


In [55]:
crop_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Practice%20Assignment/Annual_Crop_Data.csv', colClasses=c(YEAR="character"))
daily_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Practice%20Assignment/Daily_FX.csv', colClasses=c(DATE="character"))

head(crop_df)
head(daily_df)

,CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD
,<int>,<chr>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,0,1965-12-31,Barley,Alberta,1372000,1372000,2504000,1825
2,1,1965-12-31,Barley,Canada,2476800,2476800,4752900,1920
3,2,1965-12-31,Barley,Saskatchewan,708000,708000,1415000,2000
4,3,1965-12-31,Canola,Alberta,297400,297400,215500,725
5,4,1965-12-31,Canola,Canada,580700,580700,512600,885
6,5,1965-12-31,Canola,Saskatchewan,224600,224600,242700,1080


,DFX_ID,DATE,FXUSDCAD
,<int>,<chr>,<dbl>
1,0,2017-01-03,1.3435
2,1,2017-01-04,1.3315
3,2,2017-01-05,1.3244
4,3,2017-01-06,1.3214
5,4,2017-01-09,1.3240
6,5,2017-01-10,1.3213


In [39]:
dbWriteTable(conn, "CROP_DATA", crop_df, overwrite=TRUE, header = TRUE)

<h4>Write the table in table named DAILY_FX </h4>


In [40]:
# Use dbWriteTable function to write the csv into created table name DAILY_FX.
dbWriteTable(conn, "DAILY_FIX_DATA", daily_df, overwrite=TRUE, header = TRUE)

<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbWriteTable(conn, "DAILY_FX", daily_df, overwrite=TRUE, header = TRUE)
```

</details>


### Check list of tables in the present db.


In [41]:
# write your solution here.
dbListTables(conn)

[1] "CROP_DATA"      "DAILY_FIX_DATA" "DAILY_FX"

<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbListTables(conn)
```

</details>


## Now let's solve some practice problems using SQL commands:


### Find the number of rows in each table.


In [11]:
dbGetQuery(conn, 'SELECT COUNT(CD_ID) FROM CROP_DATA')

COUNT(CD_ID)
<int>
672


In [ ]:
# write your solution for table DAILY_FX.

<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbGetQuery(conn, 'SELECT COUNT(DFX_ID) FROM DAILY_FX')
```

</details>


### Query and display the first 6 rows of the crop data.


In [46]:
# write your solution here.
dbGetQuery(conn, 'SELECT* FROM CROP_DATA LIMIT 6')

CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD
<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>
0,1965-12-31,Barley,Alberta,1372000,1372000,2504000,1825
1,1965-12-31,Barley,Canada,2476800,2476800,4752900,1920
2,1965-12-31,Barley,Saskatchewan,708000,708000,1415000,2000
3,1965-12-31,Canola,Alberta,297400,297400,215500,725
4,1965-12-31,Canola,Canada,580700,580700,512600,885
5,1965-12-31,Canola,Saskatchewan,224600,224600,242700,1080


<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbGetQuery(conn, 'SELECT * FROM CROP_DATA LIMIT 6')
```

</details>


### List the types of crops in the crop dataset.


In [15]:
# write your solution here.
dbGetQuery(conn, 'SELECT DISTINCT (CROP_TYPE) FROM CROP_DATA')

CROP_TYPE
<chr>
Barley
Canola
Rye
Wheat


<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbGetQuery(conn, 'SELECT DISTINCT(CROP_TYPE) FROM CROP_DATA')
```

</details>


### Query and display the first 6 rows of the crop data for Rye.


In [17]:
# write your solution here.
dbGetQuery(conn, "SELECT*FROM CROP_DATA WHERE CROP_TYPE='Rye' LIMIT 6")

CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD
<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>
6,1965-12-31,Rye,Alberta,81000,81000,116400,1435
7,1965-12-31,Rye,Canada,323900,323900,453400,1400
8,1965-12-31,Rye,Saskatchewan,166000,166000,224000,1350
18,1966-12-31,Rye,Alberta,70000,70000,109000,1555
19,1966-12-31,Rye,Canada,293400,293400,437600,1490
20,1966-12-31,Rye,Saskatchewan,161000,161000,228600,1420


<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbGetQuery(conn, "SELECT * FROM CROP_DATA WHERE CROP_TYPE='Rye' LIMIT 6")
```

</details>


### Which crops have had an average yield greater than or equal to 3000 KG per Hectare?


In [21]:
# write your solution here.
dbGetQuery(conn, 'SELECT DISTINCT(CROP_TYPE) FROM CROP_DATA WHERE AVG_YIELD >3000' )

CROP_TYPE
<chr>
Barley
Wheat
Rye


<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbGetQuery(conn, 'SELECT DISTINCT(CROP_TYPE) 
FROM CROP_DATA 
WHERE AVG_YIELD > 3000')
```

</details>


### Find the first and last dates of each table.


In [51]:
dbGetQuery(conn, 'SELECT min(YEAR) FIRST_DATE, max(YEAR) LAST_DATE FROM CROP_DATA')

FIRST_DATE,LAST_DATE
<chr>,<chr>
1965-12-31,2020-12-31


In [56]:
# write your solution here for DAILY_FX.
dbGetQuery(conn, 'SELECT MIN(DATE) FIRST_DATE, MAX(DATE) LAST_DATE FROM DAILY_FX')


FIRST_DATE,LAST_DATE
<lgl>,<lgl>
NA,NA


<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbGetQuery(conn, 'SELECT min(DATE) FIRST_DATE, max(DATE) LAST_DATE FROM DAILY_FX')
```

</details>


### List the top 10 years of Wheat production in Saskatchewan in terms of harvested area.


In [59]:
dbGetQuery(conn, "SELECT strftime('%Y',YEAR) AS TOP_10_YRS, GEO, HARVESTED_AREA 
    FROM CROP_DATA 
    WHERE CROP_TYPE='Wheat' AND 
          GEO='Saskatchewan'
    ORDER BY HARVESTED_AREA DESC
    LIMIT 10")

TOP_10_YRS,GEO,HARVESTED_AREA
<chr>,<chr>,<int>
1986,Saskatchewan,8765400
1991,Saskatchewan,8595120
1983,Saskatchewan,8377000
1985,Saskatchewan,8357000
1990,Saskatchewan,8308200
1992,Saskatchewan,8284100
1987,Saskatchewan,8235400
1989,Saskatchewan,8097700
1984,Saskatchewan,8094000


### How many years did Barley yield at least 2000 KG per Hectare in Canada?


In [3]:
# write your solution here.
dbGetQuery(conn, "SELECT COUNT(DISTINCT(YEAR)) AS bARLEY_YRS_ABOVE_2000KG FROM CROP_DATA WHERE AVG_YIELD <2000 AND CROP_TYPE='Barley' AND Geo='Canada'")
           
           

bARLEY_YRS_ABOVE_2000KG
<int>
4


<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbGetQuery(conn, "SELECT COUNT(DISTINCT(YEAR)) AS BLY_YRS_ABOVE_2MTPH
    FROM CROP_DATA 
    WHERE AVG_YIELD > 2000 AND 
          CROP_TYPE='Barley' AND 
          GEO='Canada' ")
```

</details>


### How much farm land was seeeded with Barley in Alberta but not harvested each year since the year 2000?

Express your answer as a percentage of seeded area. What assumption might be implied in reporting this calculation?


In [5]:

dbGetQuery(conn, "SELECT strftime('%Y',YEAR) AS YEAR, GEO, CROP_TYPE,
            SEEDED_AREA, HARVESTED_AREA, 
            100*(SEEDED_AREA-HARVESTED_AREA)/SEEDED_AREA AS PCT_UNHARVESTED_AREA
            FROM CROP_DATA WHERE YEAR >= 2000 AND
            GEO = 'Alberta' AND CROP_TYPE = 'Barley'"
          )# write your solution here.


YEAR,GEO,CROP_TYPE,SEEDED_AREA,HARVESTED_AREA,PCT_UNHARVESTED_AREA
<chr>,<chr>,<chr>,<int>,<int>,<int>
2000,Alberta,Barley,2185300,1740100,20
2001,Alberta,Barley,1983000,1618700,18
2002,Alberta,Barley,2169100,1127000,48
2003,Alberta,Barley,2144800,1788700,16
2004,Alberta,Barley,1910100,1598500,16
2005,Alberta,Barley,1709800,1456900,14
2006,Alberta,Barley,1657100,1375900,16
2007,Alberta,Barley,1962700,1728000,11
2008,Alberta,Barley,1679400,1517600,9


<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbGetQuery(conn, "SELECT strftime('%Y',YEAR) AS YEAR, GEO, CROP_TYPE,
            SEEDED_AREA, HARVESTED_AREA, 
            100*(SEEDED_AREA-HARVESTED_AREA)/SEEDED_AREA AS PCT_UNHARVESTED_AREA
            FROM CROP_DATA WHERE YEAR >= 2000 AND
            GEO = 'Alberta' AND CROP_TYPE = 'Barley'")
```

</details>


### Over the last 3 calendar years of data, what was the average value of the Canadian dollar relative to the USD?


In [58]:
dbGetQuery(conn, "SELECT MIN(DATE) AS AS_OF_DATE, 
            AVG(FXUSDCAD) AS FX_DAILY_AVG_CAD 
    FROM  DAILY_FX
    WHERE DATE >= (SELECT MAX(DATE) - 3 YEARS FROM DAILY_FX)")

AS_OF_DATE,FX_DAILY_AVG_CAD
<lgl>,<lgl>
NA,NA


<details>
<summary>Click here to view/hide solution</summary>
<p>

```
dbGetQuery(conn, "SELECT MIN(DATE) AS AS_OF_DATE, 
            AVG(FXUSDCAD) AS FX_DAILY_AVG_CAD 
    FROM  DAILY_FX
    WHERE DATE >= (SELECT MAX(DATE) - 3 YEARS FROM DAILY_FX)")
```

</details>


### Use an implicit inner join to create a view of the crop data with an FX column included.

For simplicity, just use the FX values from December for each year.


In [57]:
dbGetQuery(conn, "SELECT CD_ID,YEAR ,CROP_TYPE, GEO, SEEDED_AREA, HARVESTED_AREA, PRODUCTION, AVG_YIELD, FXUSDCAD  
    FROM CROP_DATA, DAILY_FX 
    WHERE strftime('%Y',CROP_DATA.YEAR) = strftime('%Y',DAILY_FX.DATE) and strftime('%m', CROP_DATA.YEAR) = strftime('%m', DAILY_FX.DATE) LIMIT 5")

CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD,FXUSDCAD
<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>


<details><summary>Click here for the hint(s)</summary>

```R
To get the year use: strftime('%Y',CROP_DATA.YEAR) and strftime('%Y',DAILY_FX.DATE)

To get the month use: strftime('%m', CROP_DATA.YEAR) and strftime('%m', DAILY_FX.DATE)
```


<details><summary>Click here for the solution</summary>

```R
dbGetQuery(conn, "SELECT CD_ID,YEAR ,CROP_TYPE, GEO, SEEDED_AREA, HARVESTED_AREA, PRODUCTION, AVG_YIELD, FXUSDCAD  
    FROM CROP_DATA, DAILY_FX 
    WHERE strftime('%Y',CROP_DATA.YEAR) = strftime('%Y',DAILY_FX.DATE) and strftime('%m', CROP_DATA.YEAR) = strftime('%m', DAILY_FX.DATE) LIMIT 5")
```


## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Rav Ahuja </h4>
<h4> D.M.NAIDU </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                    |
| ---------- | ------- | ------------- | ------------------------------------------------------------------------------------- |
| 2022-03-03  | 0.4    | D.M.NAIDU     | Converted intial version to RSQLite                                                   |
| 2021-03-12 | 0.3     | Jeff Grossman | Cleaned up content for production                                                     |
| 2021-03-10 | 0.2     | Jeff Grossman | Added introductory and intermediate level problems and removed some advanced problems |
| 2021-03-04 | 0.1     | Jeff Grossman | Started content creation                                                              |
|            |         |               |                                                                                       |

## <h3 align="center"> © IBM Corporation 2022. All rights reserved. <h3/>
